# バックアップ＆リストア
----
Data&nbsp;Node上にあるデータのバックアップと リストアについて説明します。

## 準備

本章のコマンドを実行するための設定を行います。  

各サーバにAnsibleコマンドを発行するため、  
Ansibleを実行可能なユーザアカウントとそのSSHキーを次のセルで設定してください。

In [77]:
#elasticsearchをインストールするサーバーでansibleコマンドを実行するユーザー
USER='ansible'

#公開鍵認証を行う場合の秘密鍵のパス
#KEYPATH='/notebooks/etc/key/id_rsa'
KEYPATH = '~/.ssh/ansible_id_rsa'



セットアップ済みの環境のうちコマンドを送る先であるCoordinating(Client) Nodeのホストアドレスを、次のセルに記入して実行し、保存してください。

In [47]:
%env ES_HOST=XXX.XXX.XXX.233:9200

env: ES_HOST=XXX.XXX.XXX.233:9200


データを蓄積する先のインデックスを、次のセルを実行して保存してください。  

※特にセルの内容の書き換えは必要ありません。  
　前の章と異なるインデックス名にしているのは、この章のコマンドで投入する内容が前の章までのものと混ざらないようにするためです。

In [48]:
%env INDEX=meteorological-data-*

env: INDEX=meteorological-data-*


バックアップファイルの保存先ディレクトリを、次のセルに記入して実行し、保存してください。

In [49]:
backup_dir="/mnt/backup/my_backup"
%env BACKUP_DIR=backup_dir

env: BACKUP_DIR=/mnt/backup/my_backup


## バックアップ
バックアップ方法について説明します。

Elasticsearchのバックアップはクラスタ・index単位で取得することができます。   
詳細はElasticsearch Referenceの[Snapshot And Restore](https://www.elastic.co/guide/en/elasticsearch/reference/current/modules-snapshots.html)を参照してください。

### バックアップ対象indexの容量確認
バックアップの保存前に、対象となるindexが保存先のストレージに収まる容量かを確認します。

indexの容量はcat APIにより確認できます。  
詳細はElasticsearch Referenceの[cat-indices](https://www.elastic.co/guide/en/elasticsearch/reference/current/cat-indices.html)を参照してください。

In [50]:
%%bash
curl -XGET "http://$ES_HOST/_cat/indices?v&s=index"

health status index          uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   sample_index   NCA-vz62SfisSEoiBBdyHA   5   1        110            0    907.1kb        453.5kb
green  open   reindex_source FTQ0QcHxR9qVpt_sART-YQ   5   1          1            0      8.9kb          4.4kb


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   330  100   330    0     0  24422      0 --:--:-- --:--:-- --:--:-- 25384


### リポジトリの作成（初回のみ）
保存先であるリポジトリを作成します。

バックアップ用ディレクトリを作成します。
オーナーとグループは "elasticsearch"を指定します。

In [69]:
!ansible-playbook playbooks/make_backup_directory.yml --extra-vars "backup_dir=$backup_dir" -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [192.168.122.80]

TASK [cd_mnt] ******************************************************************
changed: [192.168.122.80]

TASK [mkdir] *******************************************************************
 [WARNING]: Consider using file module with state=directory rather than running
mkdir

changed: [192.168.122.80]

TASK [chown] *******************************************************************
 [WARNING]: Consider using file module with owner rather than running chown

changed: [192.168.122.80]

PLAY RECAP *********************************************************************
192.168.122.80             : ok=4    changed=3    unreachable=0    failed=0   



elasticsearch.ymlにバックアップ用ディレクトリパスを設定します。

In [70]:
!ansible-playbook playbooks/set_backup_directory.yml --extra-vars "backup_dir=$backup_dir" -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [192.168.122.80]

TASK [set_shared file system repository] ***************************************
ok: [192.168.122.80]

PLAY RECAP *********************************************************************
192.168.122.80             : ok=2    changed=0    unreachable=0    failed=0   



elasticsearchを再起動します。

In [59]:
!ansible-playbook playbooks/restart_cluster.yml -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [192.168.122.80]

TASK [restart elasticsearch] ***************************************************
changed: [192.168.122.80]

PLAY RECAP *********************************************************************
192.168.122.80             : ok=2    changed=1    unreachable=0    failed=0   



Elasticsearchでバックアップを保存するには、データのスナップショットを保存する先であるリポジトリを作成する必要があります。  
my_backupという名前のリポジトリを作るコマンドは次の通りです。

In [79]:
%%bash
curl -XPUT http://$ES_HOST/_snapshot/my_backup?pretty -H 'Content-Type:application/json' -d @- << EOF
{
    "type": "fs",
    "settings": {
        "location": "$BACKUP_DIR",
        "compress": true
    }
}
EOF

{
  "acknowledged" : true
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   137  100    28  100   109    166    648 --:--:-- --:--:-- --:--:--   652


my_backupリポジトリが作成されたことを確認します

In [80]:
%%bash
curl -XGET "http://$ES_HOST/_snapshot/my_backup?pretty"

{
  "my_backup" : {
    "type" : "fs",
    "settings" : {
      "compress" : "true",
      "location" : "/mnt/backup/my_backup"
    }
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   140  100   140    0     0   8829      0 --:--:-- --:--:-- --:--:--  9333


### バックアップ実行
実際にデータをバックアップします。  
クラスタ全体で取るのか、index単位で取るのかでコマンドを切り替えてください。


オプションについては、Elasticsearch Referenceの[Snapshot And Restore](https://www.elastic.co/guide/en/elasticsearch/reference/current/modules-snapshots.html)を参照してください。

**■クラスタ全体をバックアップ**

クラスタ全体のバックアップを、my_backupリポジトリにsnapshot_1という名前で保存するコマンドは次の通りです。

In [74]:
%%bash
curl -XPUT "http://$ES_HOST/_snapshot/my_backup/snapshot_1?wait_for_completion=true&pretty"

{
  "snapshot" : {
    "snapshot" : "snapshot_1",
    "uuid" : "3T5V4TwsQ7C1a_O0cqO7NA",
    "version_id" : 5000099,
    "version" : "5.0.0",
    "indices" : [
      "sample_index",
      "meteorological-data-2015.07",
      "meteorological-data-2015.08",
      "meteorological-data-2015.03",
      "meteorological-data-2015.09",
      "meteorological-data-2015.05",
      "meteorological-data-2015.01",
      "meteorological-data-2015.04",
      "meteorological-data-2015.12",
      "meteorological-data-2015.11",
      "meteorological-data-2015.10",
      "meteorological-data-2015.02",
      "meteorological-data-2015.06",
      "meteorological-data-2014.12"
    ],
    "state" : "SUCCESS",
    "start_time" : "2016-12-13T04:31:41.470Z",
    "start_time_in_millis" : 1481603501470,
    "end_time" : "2016-12-13T04:31:42.079Z",
    "end_time_in_millis" : 1481603502079,
    "duration_in_millis" : 609,
    "failures" : [ ],
    "shards" : {
      "total" : 70,
      "failed" : 0,
      "successful

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1019  100  1019    0     0   1555      0 --:--:-- --:--:-- --:--:--  1555


**■インデックスを指定してバックアップ**

meteorological-data-\* indexのバックアップを、my_backupリポジトリにsnapshot_2という名前で保存するコマンドは次の通りです。

In [75]:
%%bash
curl -XPUT "http://$ES_HOST/_snapshot/my_backup/snapshot_2" -H 'Content-Type:application/json' -d @- << EOF
{
  "indices": "$INDEX",
  "ignore_unavailable": true,
  "include_global_state": false
}
EOF

{"accepted":true}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   116  100    17  100    99    153    891 --:--:-- --:--:-- --:--:--   900


### バックアップできたかの確認
リポジトリの内容を閲覧して、バックアップが取れたかを確認します。

snapshot_1という名前のバックアップ内容を確認するコマンドは次の通りです。

In [76]:
%%bash
curl -XGET "http://$ES_HOST/_snapshot/my_backup/snapshot_1?pretty"

{
  "snapshots" : [
    {
      "snapshot" : "snapshot_1",
      "uuid" : "3T5V4TwsQ7C1a_O0cqO7NA",
      "version_id" : 5000099,
      "version" : "5.0.0",
      "indices" : [
        "sample_index",
        "meteorological-data-2015.07",
        "meteorological-data-2015.08",
        "meteorological-data-2015.03",
        "meteorological-data-2015.09",
        "meteorological-data-2015.05",
        "meteorological-data-2015.01",
        "meteorological-data-2015.04",
        "meteorological-data-2015.12",
        "meteorological-data-2015.11",
        "meteorological-data-2015.10",
        "meteorological-data-2015.02",
        "meteorological-data-2015.06",
        "meteorological-data-2014.12"
      ],
      "state" : "SUCCESS",
      "start_time" : "2016-12-13T04:31:41.470Z",
      "start_time_in_millis" : 1481603501470,
      "end_time" : "2016-12-13T04:31:42.079Z",
      "end_time_in_millis" : 1481603502079,
      "duration_in_millis" : 609,
      "failures" : [ ],
      "shards

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1096  100  1096    0     0  72823      0 --:--:-- --:--:-- --:--:-- 78285


## リストア
リストア方法について説明します。

### リストア先のindex状況の確認
データリストアはindexが空のところに入れることを前提としています。  
リストア実施前に、同名indexが存在しないことを確認してください。 

現状存在するindexを確認するコマンドは、次の通りです。  
詳細はElasticsearch Referenceの[cat-indices](https://www.elastic.co/guide/en/elasticsearch/reference/current/cat-indices.html)を参照してください。

In [67]:
%%bash
curl -XGET "http://$ES_HOST/_cat/indices?v&s=index"

health status index          uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   reindex_dest   hm0itWDWTP-h05dMnJSRAQ   5   1          1            0      8.8kb          4.4kb
green  open   reindex_source 0CVwNJ0XQievecjHPq5kNQ   5   1          1            0      8.9kb          4.4kb
green  open   sample_index   NCA-vz62SfisSEoiBBdyHA   5   1        110            0    907.1kb        453.5kb


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   440  100   440    0     0  28299      0 --:--:-- --:--:-- --:--:-- 29333


また、snapshotから一部のindexをリストアすることやindex名を変更しながらリストアすることも可能ですので、  
状況に応じて必要なindexをリストアしてください。    
詳細はElasticsearch Referenceの[Restoring from a Snapshotedit](https://www.elastic.co/guide/en/elasticsearch/reference/current/modules-snapshots.html)を参照してください。

### リストア実行
実際にデータをリストアします。

例として、[バックアップ実行](#バックアップ実行)で取得したバックアップ(snapshot_1)からリストアするコマンドは次の通りです。

In [68]:
%%bash
curl -XPOST "http://$ES_HOST/_snapshot/my_backup/snapshot_1/_restore?pretty"

{
  "accepted" : true
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    24  100    24    0     0    239      0 --:--:-- --:--:-- --:--:--   242
